In [580]:
# importing necessary libraries
import json
import os
import pandas as pd
import numpy as np
import time
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score





In [581]:
#converting the json data to pandas dataframe
files = ['transactions1.json', 'transactions2.json', 'transactions3.json']
combined_data = {
    'deposits': [],
    'borrows': [],
    'withdraws': [],
    'repays': [],
    'liquidates': []
}
for filename in files:
    file_path = os.path.join("transactions", filename)  
    with open(file_path, 'r') as f:
        data = json.load(f)
        for key in combined_data.keys():
            if key in data:
                combined_data[key].extend(data[key])

In [582]:
#visulaizing each transaction type

deposits_df = pd.json_normalize(combined_data['deposits'])
borrows_df = pd.json_normalize(combined_data['borrows'])
withdraws_df = pd.json_normalize(combined_data['withdraws'])
repays_df = pd.json_normalize(combined_data['repays'])
liquidates_df = pd.json_normalize(combined_data['liquidates'])


In [583]:
deposits_df['transaction_type'] = 'deposit'
borrows_df['transaction_type'] = 'borrow'
withdraws_df['transaction_type'] = 'withdraw'
repays_df['transaction_type'] = 'repay'
liquidates_df['transaction_type'] = 'liquidate'


In [584]:
transactions_df = pd.concat([
    deposits_df,
    borrows_df,
    withdraws_df,
    repays_df,
    liquidates_df
], ignore_index=True)


In [585]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127103 entries, 0 to 127102
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   amount            127103 non-null  object
 1   amountUSD         127103 non-null  object
 2   hash              127103 non-null  object
 3   id                127103 non-null  object
 4   timestamp         127103 non-null  object
 5   account.id        117103 non-null  object
 6   asset.id          127103 non-null  object
 7   asset.symbol      127103 non-null  object
 8   transaction_type  127103 non-null  object
 9   liquidatee.id     10000 non-null   object
 10  liquidator.id     10000 non-null   object
dtypes: object(11)
memory usage: 10.7+ MB


In [586]:
# inlcluding the liquidee wallet in liquiditee olumn and the liquidator wallet in account id column
transactions_df['liquidatee.id'] = transactions_df.apply(
    lambda row: row['liquidatee.id'] if row['transaction_type'] == 'liquidate' else '0x0000000000000000000000000000000000000000',
    axis=1
)
transactions_df['account.id'] = transactions_df.apply(
    lambda row: row['liquidator.id'] if row['transaction_type'] == 'liquidate' else row['account.id'],
    axis=1
)



In [587]:
# Keep only the useful columns
transactions_df = transactions_df[['account.id','amount', 'amountUSD', 'asset.symbol', 'transaction_type','timestamp', 'liquidatee.id']]
transactions_df = transactions_df.sort_values(by='liquidatee.id', ascending=False)
transactions_df.head(10)

,account.id,amount,amountUSD,asset.symbol,transaction_type,timestamp,liquidatee.id
123831,0xf30087bbf02155abda1a162a29e97fcc41370288,355136738,107.1709095780869864185980200886035,DAI,liquidate,1661592224,0xffec5ea11fa72dcfac5ae54da1778d9db46fcade
126507,0xbfe1364efb0a1e01241f058a6039f43fea88cfc5,877789099861,30.16279769792983601343868417614317,DAI,liquidate,1723263539,0xffeb9214f614c763f429809956328069e781fe6d
126652,0x681d0d7196a036661b354fa2a7e3b73c2adc43ec,952246431316,32.93515403999549468504572329975496,USDC,liquidate,1723599179,0xffdc0c7dff4bdabc76e69f1ffd0c11d54b21369b
126632,0xbfe1364efb0a1e01241f058a6039f43fea88cfc5,52429202095,65.86343603885500470438528914725027,USDC,liquidate,1723523939,0xffdc0c7dff4bdabc76e69f1ffd0c11d54b21369b
124655,0xe6e1d841b873f02d072b902c5100f3f10bf4f4e2,832747903818,184.0438346399537008827001192438712,USDC,liquidate,1668268607,0xffdc0c7dff4bdabc76e69f1ffd0c11d54b21369b
125391,0x796d37daf7cdc455e023be793d0daa6240707069,17975673,100.0900658591347615545190368683597,USDC,liquidate,1696567499,0xffdc0c7dff4bdabc76e69f1ffd0c11d54b21369b
120866,0xd911560979b78821d7b045c79e36e9cbfc2f6c6f,5317725219,1389.589313754203843987983729065251,USDC,liquidate,1655106044,0xffbfe38b14b6e1d9ed7f43cac6739ba27bf61200
120346,0xd911560979b78821d7b045c79e36e9cbfc2f6c6f,9529776820,2751.513257720762199007336327699278,USDC,liquidate,1655011618,0xffbfe38b14b6e1d9ed7f43cac6739ba27bf61200
126032,0xa98cee9c499905808878dbd582d1ea9423f16436,17710576588478,4183.632161012845866774028406908658,MKR,liquidate,1721187443,0xff789ab6730c22da1e095645836cb70eaf5cfc51
122476,0xd911560979b78821d7b045c79e36e9cbfc2f6c6f,10821455842,2080.44684390924431800049075997616,DAI,liquidate,1655575622,0xff61d6ad66b77f2e5218db123095a14d7247fe64


In [588]:
#checking for null values in the dataset
transactions_df.isnull().sum()

account.id          0
amount              0
amountUSD           0
asset.symbol        0
transaction_type    0
timestamp           0
liquidatee.id       0
dtype: int64

In [589]:
#converting the timestamp to datetime format
transactions_df['datetime'] = pd.to_datetime(transactions_df['timestamp'], unit='s')


C:\Users\hp\AppData\Local\Temp\ipykernel_22912\2851813292.py:2: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  transactions_df['datetime'] = pd.to_datetime(transactions_df['timestamp'], unit='s')


In [590]:
# Final check
print(transactions_df.info())
print(transactions_df.shape)


<class 'pandas.core.frame.DataFrame'>
Index: 127103 entries, 123831 to 63551
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   account.id        127103 non-null  object        
 1   amount            127103 non-null  object        
 2   amountUSD         127103 non-null  object        
 3   asset.symbol      127103 non-null  object        
 4   transaction_type  127103 non-null  object        
 5   timestamp         127103 non-null  object        
 6   liquidatee.id     127103 non-null  object        
 7   datetime          127103 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(7)
memory usage: 8.7+ MB
None
(127103, 8)


In [591]:
def safe_to_float(val):
    try:
        return float(val)
    except:
        return 0  # or 0 if you prefer

transactions_df['amount'] = transactions_df['amount'].apply(safe_to_float)
transactions_df['amountUSD'] = transactions_df['amountUSD'].apply(safe_to_float)

# Drop rows where conversion failed
transactions_df.dropna(subset=['amount', 'amountUSD'], inplace=True)
transactions_df['timestamp'] = transactions_df['timestamp'].astype(int)


In [592]:
# grouping the data by account id and transaction type
total_deposited = transactions_df[transactions_df['transaction_type'] == 'deposit'] \
    .groupby('account.id',sort=False)['amountUSD'].sum().reset_index() \
    .rename(columns={'amountUSD': 'total_deposited'})

total_borrowed = transactions_df[transactions_df['transaction_type'] == 'borrow'] \
    .groupby('account.id',sort=False)['amountUSD'].sum().reset_index() \
    .rename(columns={'amountUSD': 'total_borrowed'})

total_withdrawn = transactions_df[transactions_df['transaction_type'] == 'withdraw'] \
    .groupby('account.id',sort=False)['amountUSD'].sum().reset_index() \
    .rename(columns={'amountUSD': 'total_withdrawn'})

total_repaid = transactions_df[transactions_df['transaction_type'] == 'repay'] \
    .groupby('account.id',sort=False)['amountUSD'].sum().reset_index() \
    .rename(columns={'amountUSD': 'total_repaid'})

In [593]:
#wallet age and last transaction time
# Calculate wallet age in days  

all_tx = transactions_df[['account.id', 'timestamp']]
wallet_age_df = all_tx.groupby('account.id',sort=False)['timestamp'].min().reset_index()
wallet_age_df['wallet_age'] = (int(time.time()) - wallet_age_df['timestamp']) / (60 * 60 * 24)

# 1. Convert timestamp to datetime (use correct unit: 's' for seconds, 'ms' for milliseconds)
transactions_df['datetime'] = pd.to_datetime(transactions_df['timestamp'], unit='s')

# 2. Get the last transaction time per wallet
last_txn = transactions_df.groupby('account.id')['datetime'].max().reset_index()
last_txn.columns = ['account.id', 'last_txn_time']

# 3. Use a reference date — for example, the latest date in your dataset
reference_date = transactions_df['datetime'].max()

# 4. Compute time since last transaction in days
last_txn['time_since_last_tx'] = (reference_date - last_txn['last_txn_time']).dt.total_seconds() / (60 * 60 * 24)




In [594]:
#calculate the count of liquidations per account
# #calculate liquidation risk ratio by standard deviation and average of the total liquidated amount
liquidations_df = transactions_df[transactions_df['transaction_type'] == 'liquidate']

liquidatee_counts = liquidations_df.groupby('liquidatee.id',sort=False).size().reset_index(name='liquidatee_count') \
    .rename(columns={'liquidatee.id': 'account.id'})
liquidator_counts = liquidations_df.groupby('account.id',sort=False).size().reset_index(name='liquidator_count')
total_liquidated = liquidations_df.groupby('liquidatee.id',sort=False)['amountUSD'].sum().reset_index() \
    .rename(columns={'liquidatee.id': 'account.id', 'amountUSD': 'total_liquidated'})
avg_liq = total_liquidated['total_liquidated'].mean()
std_liq = total_liquidated['total_liquidated'].std()
total_liquidated['liquidation_risk_ratio'] = (total_liquidated['total_liquidated'] - avg_liq) / std_liq
total_liquidated['liquidation_risk_ratio'] = total_liquidated['liquidation_risk_ratio'].fillna(0)


In [595]:

#merging all the dataframes to create a single dataframe for features
features_df = total_deposited \
    .merge(total_borrowed, on='account.id', how='outer',sort=False) \
    .merge(total_withdrawn, on='account.id', how='outer',sort=False) \
    .merge(total_repaid, on='account.id', how='outer',sort=False) \
    .merge(wallet_age_df[['account.id', 'wallet_age']], on='account.id', how='outer',sort=False) \
    .merge(last_txn[['account.id', 'time_since_last_tx']], on='account.id', how='left',sort=False) \
    .merge(liquidatee_counts, on='account.id', how='outer',sort=False) \
    .merge(liquidator_counts, on='account.id', how='outer',sort=False) \
    .merge(total_liquidated, on='account.id', how='left',sort=False)




In [596]:
features_df[['total_liquidated', 'liquidation_risk_ratio']] = features_df[
    ['total_liquidated', 'liquidation_risk_ratio']
].fillna(0)

In [597]:
features_df.head(10)

,account.id,total_deposited,total_borrowed,total_withdrawn,total_repaid,wallet_age,time_since_last_tx,liquidatee_count,liquidator_count,total_liquidated,liquidation_risk_ratio
0,0x00000000001876eb1444c986fd502e618c587430,35117.726303,NaN,1.117321e+06,NaN,1789.084664,1640.138762,NaN,NaN,0.0,0.0
1,0x000000000025d4386f7fb58984cbe110aee3a4c4,NaN,NaN,8.971524e+03,NaN,1726.804653,1647.004363,NaN,NaN,0.0,0.0
2,0x000000000081105ffd4392520dd13c4c70d95d19,51.142175,NaN,NaN,NaN,1779.698380,1726.923553,NaN,NaN,0.0,0.0
3,0x00000000008943c65caf789fffcf953be156f6f8,0.010000,NaN,2.311855e+04,NaN,1781.143854,1651.620845,NaN,NaN,0.0,0.0
4,0x0000000038355af6ffd5328a16cfd2170e59f39c,NaN,NaN,NaN,2059.748569,536.776076,443.971528,NaN,4.0,0.0,0.0
5,0x00000000553a85582988aa8ad43fb7dda2466bc7,NaN,NaN,1.085416e+03,NaN,1694.860822,1642.085995,NaN,NaN,0.0,0.0
6,0x000000005bcf85aad6ed1d32db5490deddfc97f9,NaN,NaN,NaN,9798.584874,511.107326,227.719583,NaN,22.0,0.0,0.0
7,0x00000000af5a61acaf76190794e3fdf1289288a1,NaN,NaN,3.481944e+03,50724.344572,2090.894352,1905.568831,NaN,NaN,0.0,0.0
8,0x00000000b1786c9698c160d78232c78d6f6474fe,NaN,NaN,5.373094e+05,NaN,1717.375764,1649.532465,NaN,NaN,0.0,0.0
9,0x0000000484f2217f1a64eb6d24b5cee446faeae5,416571.956590,NaN,2.372816e+06,NaN,1784.888843,1646.713125,NaN,NaN,0.0,0.0


In [598]:
def nz_mean_std(series):
    nz = series[series > 0]
    return nz.mean(), nz.std()
def compute_liquidation_penalty(row):
    penalty = 0
    
    # Dynamic penalty for liquidatee count (if it's above a threshold, apply a penalty)
    liquidatee_count_thresholds = {1: 0, 2: 1, 3: 2, 4: 3}  # Example thresholds (more counts = higher penalty)
    if row['liquidatee_count'] > 0:
        penalty += liquidatee_count_thresholds.get(row['liquidatee_count'], 3)  # Apply penalty based on count

    # Dynamic penalty for liquidator count (if it's above a threshold, apply a penalty)
    liquidator_count_thresholds = {1: 0, 2: 1, 3: 2, 4: 3}
    if row['liquidator_count'] > 0:
        penalty += liquidator_count_thresholds.get(row['liquidator_count'], 3)

    # Liquidation risk ratio (dynamic deduction based on the value)
    if row['liquidation_risk_ratio'] > 0:
        penalty += 1  # Start with some penalty for any liquidation risk ratio above 0
    if row['liquidation_risk_ratio'] > 3:
        penalty += 2  # Additional penalty for liquidation risk ratio above 3

    return penalty



In [599]:


# Function to calculate the credit score
def compute_credit_score(row):
    score = 50  # Initial score
    
    # Get the mean and standard deviation for each feature
    avg_dep, std_dep = nz_mean_std(features_df['total_deposited'])
    avg_bor, std_bor = nz_mean_std(features_df['total_borrowed'])
    avg_rep, std_rep = nz_mean_std(features_df['total_repaid'])
    avg_wdr, std_wdr = nz_mean_std(features_df['total_withdrawn'])
    avg_age, std_age = nz_mean_std(features_df['wallet_age'])
    avg_last_tx, std_last_tx = nz_mean_std(features_df['time_since_last_tx'])
    
    # Handle missing values by checking for NaN and replacing with 0
    total_deposited = row['total_deposited'] if not np.isnan(row['total_deposited']) else 0
    total_borrowed = row['total_borrowed'] if not np.isnan(row['total_borrowed']) else 0
    total_repaid = row['total_repaid'] if not np.isnan(row['total_repaid']) else 0
    total_withdrawn = row['total_withdrawn'] if not np.isnan(row['total_withdrawn']) else 0
    wallet_age = row['wallet_age'] if not np.isnan(row['wallet_age']) else 0
    time_since_last_tx = row['time_since_last_tx'] if not np.isnan(row['time_since_last_tx']) else 0
    
    # Prevent division by zero by checking if std is 0, and set it to 1 if so
    std_dep = std_dep if std_dep != 0 else 1
    std_bor = std_bor if std_bor != 0 else 1
    std_rep = std_rep if std_rep != 0 else 1
    std_wdr = std_wdr if std_wdr != 0 else 1
    std_age = std_age if std_age != 0 else 1
    std_last_tx = std_last_tx if std_last_tx != 0 else 1
    
    # Calculate score increments based on feature comparison
    if total_deposited > avg_dep:
        score += ((total_deposited - avg_dep) / std_dep)/10 # Normalized by std
    
    if total_borrowed > avg_bor:
        score -= ((total_borrowed - avg_bor) / std_bor)/10 # Normalized by std
    
    if total_repaid > avg_rep:
        score += ((total_repaid - avg_rep) / std_rep)/10 # Normalized by std
    
    if total_withdrawn > avg_wdr:
        score -= ((total_withdrawn - avg_wdr) / std_wdr)/10 # Normalized by std
    
    if wallet_age > avg_age:
        score += (wallet_age - avg_age) / std_age
    elif wallet_age < 30:
        score -= 2
    
    if time_since_last_tx > avg_last_tx:
        score += (time_since_last_tx - avg_last_tx) / std_last_tx
    elif time_since_last_tx < avg_last_tx:
        score -= (avg_last_tx - time_since_last_tx) / std_last_tx
    
    # Apply liquidation penalty (if any)
    penalty = compute_liquidation_penalty(row)
    score -= penalty
    
    # Return the raw score
    return max(0, score)  # Ensure score is between 0 and 100

# Calculate raw credit score for each row







In [600]:
#create the credit score column in the features dataframe
features_df['credit_score'] = features_df.apply(compute_credit_score, axis=1)


In [601]:
#normalizing the credit score to be between 0 and 100
min_score = features_df['credit_score'].min()
max_score = features_df['credit_score'].max()

features_df['credit_score'] = features_df['credit_score'].apply(
    lambda x: (x - min_score) / (max_score - min_score) * 100
)



In [602]:
features_df.head()

,account.id,total_deposited,total_borrowed,total_withdrawn,total_repaid,wallet_age,time_since_last_tx,liquidatee_count,liquidator_count,total_liquidated,liquidation_risk_ratio,credit_score
0,0x00000000001876eb1444c986fd502e618c587430,35117.726303,NaN,1.117321e+06,NaN,1789.084664,1640.138762,NaN,NaN,0.0,0.0,65.822151
1,0x000000000025d4386f7fb58984cbe110aee3a4c4,NaN,NaN,8.971524e+03,NaN,1726.804653,1647.004363,NaN,NaN,0.0,0.0,65.014904
2,0x000000000081105ffd4392520dd13c4c70d95d19,51.142175,NaN,NaN,NaN,1779.698380,1726.923553,NaN,NaN,0.0,0.0,66.939795
3,0x00000000008943c65caf789fffcf953be156f6f8,0.010000,NaN,2.311855e+04,NaN,1781.143854,1651.620845,NaN,NaN,0.0,0.0,65.921685
4,0x0000000038355af6ffd5328a16cfd2170e59f39c,NaN,NaN,NaN,2059.748569,536.776076,443.971528,NaN,4.0,0.0,0.0,28.421644


In [603]:
# Prepare features and target
X = features_df.drop(['account.id', 'credit_score'], axis=1)  # Drop non-numeric columns
y = features_df['credit_score']

# Handle missing values (if any)
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)

# Split the full dataset into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest model

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Make predictions
y_pred = rf.predict(X_test)

# Evaluate the model

mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae:.2f}')
mean_squared_error = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mean_squared_error:.2f}')
r2 = r2_score(y_test, y_pred)   
print(f'R^2 Score: {r2:.2f}')



Mean Absolute Error: 0.07
Mean Squared Error: 0.50
R^2 Score: 1.00


In [607]:
# Reset index for X_test and predicted values
X_test_reset = X_test.reset_index()

# Add predictions
X_test_reset['credit_score'] = y_pred

# Merge with account IDs from the original DataFrame
account_ids = features_df[['account.id']].reset_index()
merged_df = account_ids.merge(X_test_reset, left_on='index', right_on='index', how='right')

# Drop the index column as it's no longer needed
merged_df.drop(columns=['index'], inplace=True)

# Limit to 1000 entries (or fewer if less available)
final_1000 = merged_df.head(1000)

# View or use the DataFrame
final_1000_sorted = final_1000.sort_values(by='credit_score',ascending=False)


# If you'd like to export:
final_1000_sorted.to_csv("predicted_credit_scores_1000.csv", index=False)


In [606]:
final_1000_sorted.head(1000)

,account.id,total_deposited,total_borrowed,total_withdrawn,total_repaid,wallet_age,time_since_last_tx,liquidatee_count,liquidator_count,total_liquidated,liquidation_risk_ratio,credit_score
544,0x3ba21b6477f48273f41d241aa3722ffb9e07e247,5.871014e+07,0.0,5.403252e+08,0.0,1781.910417,1668.804491,0.0,0.0,0.000000,0.000000,80.735155
721,0x1a7e4c7ce76f0d744f5bbf4239b01d58aafb2483,2.076551e+02,0.0,0.000000e+00,0.0,2130.805463,2078.030637,0.0,0.0,0.000000,0.000000,77.238469
39,0x0e58838b582c3964d6a6713951a979b36c75d16c,5.827970e+00,0.0,0.000000e+00,0.0,2130.093970,2077.319144,0.0,0.0,0.000000,0.000000,77.212792
422,0x8fdd0cf22012a5fecdbf77ef30d9e9834dc1bf0a,1.747585e+02,0.0,0.000000e+00,0.0,2131.949051,2074.237176,0.0,0.0,0.000000,0.000000,77.201095
566,0xa7171cc94e1511632715bda67de086c0d83f5402,7.847714e+04,0.0,0.000000e+00,0.0,2130.756146,2073.497164,0.0,0.0,0.000000,0.000000,77.175791
...,...,...,...,...,...,...,...,...,...,...,...,...
710,0x8bdc3d98a267020f80936e4ff18f279b611452c3,0.000000e+00,0.0,0.000000e+00,0.0,0.000000,0.000000,4.0,0.0,4861.836678,-0.086246,9.394820
851,0x8ab7b0767080cb6f5391a7f2cf2bb1506e1789ef,0.000000e+00,0.0,0.000000e+00,0.0,0.000000,0.000000,6.0,0.0,2243.035554,-0.087538,9.394820
183,0x356614d974a439947d1f92b6e59a675c5028db39,0.000000e+00,0.0,0.000000e+00,0.0,0.000000,0.000000,6.0,0.0,1989.522773,-0.087663,9.394820
481,0x1c3d205e0cb815fdb7183fb4024db44762db8c06,0.000000e+00,0.0,0.000000e+00,0.0,0.000000,0.000000,4.0,0.0,3138.899705,-0.087096,9.394820
